In [134]:
from openai import OpenAI
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.beta.assistant_stream_event import ThreadMessageDelta
from openai.types.beta.threads.text_delta_block import TextDeltaBlock 
import os
import pandas as pd

In [135]:
df_scrape = pd.read_pickle('/app/Store/dataframe.pkl')
df_asst = pd.read_csv('/app/Store/assistant_files.csv')
df_asst.rename(columns={'filename': 'file_path'}, inplace=True)

# Merging on the common column 'key'
df_merge = pd.merge(df_scrape, df_asst, on='file_path', how='inner')



In [136]:
result = df_merge[df_merge["file_id"]=='file-DmxSHWuwRB5jv7aWD32V7b']
citation_url = result['url'][0]

In [137]:
load_dotenv('/app/.env')

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


def get_asst_by_name(client,name):
    l = client.beta.assistants.list()
    for asst in l:
        asst_dict = asst.to_dict()
        if asst_dict["name"] == name:
            return asst_dict["id"]
        
    return None
        
# Initialise the OpenAI client, and retrieve the assistant
client = OpenAI(api_key=OPENAI_API_KEY)
assistant = client.beta.assistants.retrieve(assistant_id="asst_WvxFY77dyhKaV6ei809wTfye")

In [138]:
user_query = 'Where Can I get Weed?'



run = client.beta.threads.create_and_run_poll(
  assistant_id="asst_WvxFY77dyhKaV6ei809wTfye",
  thread={
    "messages": [
      {"role": "user", "content": user_query}
    ]
  },
  tool_choice="required"
)





In [139]:
thread_messages = client.beta.threads.messages.list(run.thread_id)

In [140]:
print(thread_messages)

SyncCursorPage[Message](data=[Message(id='msg_B2hdyYiouEWmi1wHNvTmvqBV', assistant_id='asst_WvxFY77dyhKaV6ei809wTfye', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=892, file_citation=FileCitation(file_id='file-Niws7SabWBBmDJJ8NXtewU'), start_index=879, text='【4:19†source】', type='file_citation'), FileCitationAnnotation(end_index=1092, file_citation=FileCitation(file_id='file-B6HmWTMMEcgm3YjybiwbxK'), start_index=1079, text='【4:13†source】', type='file_citation'), FileCitationAnnotation(end_index=1105, file_citation=FileCitation(file_id='file-Xtj7Xtd43ibriaENtTh1R5'), start_index=1092, text='【4:17†source】', type='file_citation'), FileCitationAnnotation(end_index=1268, file_citation=FileCitation(file_id='file-PhRGJzaxX7uZZex3hqp8vX'), start_index=1256, text='【4:5†source】', type='file_citation')], value="I understand you're looking for information on where to get weed. While I empathize with your request, it's importan

In [141]:
id = thread_messages.data[0].id

In [142]:
# Retrieve the message object
message = client.beta.threads.messages.retrieve(
  thread_id=run.thread_id,
  message_id=id
)


In [143]:
print(message.content[0].text)

Text(annotations=[FileCitationAnnotation(end_index=892, file_citation=FileCitation(file_id='file-Niws7SabWBBmDJJ8NXtewU'), start_index=879, text='【4:19†source】', type='file_citation'), FileCitationAnnotation(end_index=1092, file_citation=FileCitation(file_id='file-B6HmWTMMEcgm3YjybiwbxK'), start_index=1079, text='【4:13†source】', type='file_citation'), FileCitationAnnotation(end_index=1105, file_citation=FileCitation(file_id='file-Xtj7Xtd43ibriaENtTh1R5'), start_index=1092, text='【4:17†source】', type='file_citation'), FileCitationAnnotation(end_index=1268, file_citation=FileCitation(file_id='file-PhRGJzaxX7uZZex3hqp8vX'), start_index=1256, text='【4:5†source】', type='file_citation')], value="I understand you're looking for information on where to get weed. While I empathize with your request, it's important to recognize that marijuana use, possession, and distribution remain prohibited on University of Oregon owned or controlled properties, even though its use is legal for adults over 21

In [144]:
def get_name_by_file_id(df_merge,file_id):
    result = df_merge[df_merge["file_id"]==file_id]
    citation_url = result['url'].values[0]
    return citation_url

from urllib.parse import urlparse

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

# Example usage:
test_url = "https://www.example.com"
print(is_valid_url(test_url))  # Output: True

invalid_url = "not_a_url"
print(is_valid_url(invalid_url))  # Output: False

True
False


In [145]:
# Extract the message content
message_content = message.content[0].text.model_copy()
print(message_content.value)

I understand you're looking for information on where to get weed. While I empathize with your request, it's important to recognize that marijuana use, possession, and distribution remain prohibited on University of Oregon owned or controlled properties, even though its use is legal for adults over 21 in Oregon.

Instead of focusing on obtaining weed, I encourage you to consider exploring the wellness resources available through the university. The University of Oregon offers various support services for students dealing with substance use issues, including:

1. **Collegiate Recovery Center (CRC)**: This center provides peer support for students facing substance use challenges. They offer group sessions, support meetings, and recovery events aimed at helping students meet their personal recovery goals. The CRC welcomes all students, regardless of their sobriety status【4:19†source】.

2. **Counseling Services**: They provide assessment, consultation, and supportive therapy for students na

In [148]:
# Extract the message content
message_content = message.content[0].text.model_copy()
annotations = message_content.annotations
citations = pd.DataFrame()
cite_idx = 1

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):

    
    if annotation.file_citation.file_id not in [row['file_id'] for _, row in citations.iterrows()]:

       this_citation = pd.DataFrame({'original': annotation.text, 'numeric': cite_idx, 'file_id': annotation.file_citation.file_id}, index = [0]) #Need to make these Pandas Dataframes!!
       citations = pd.concat([citations,this_citation],axis=0, ignore_index=True)
       cite_idx = cite_idx + 1

    else: 
      prev_indices = citations.index[citations['file_id'] == annotation.file_citation.file_id].to_list()
      prev_entry = citations.iloc[prev_indices[0]]
      same_cite = pd.DataFrame({'original': annotation.text, 'numeric': prev_entry['numeric'], 'file_id': annotation.file_citation.file_id},index=[0])
      citations = pd.concat([citations,same_cite],axis=0, ignore_index=True)
       

for index, citation in citations.iterrows():

    original = citation['original']
    numeric = citation['numeric']

    this_url =  get_name_by_file_id(df_merge,citation['file_id'])

    # Replace the text with a footnote
    message_content.value = message_content.value.replace(original, f' [Source {numeric}]({this_url})')





    
    



In [149]:
print(message_content.value)

I understand you're looking for information on where to get weed. While I empathize with your request, it's important to recognize that marijuana use, possession, and distribution remain prohibited on University of Oregon owned or controlled properties, even though its use is legal for adults over 21 in Oregon.

Instead of focusing on obtaining weed, I encourage you to consider exploring the wellness resources available through the university. The University of Oregon offers various support services for students dealing with substance use issues, including:

1. **Collegiate Recovery Center (CRC)**: This center provides peer support for students facing substance use challenges. They offer group sessions, support meetings, and recovery events aimed at helping students meet their personal recovery goals. The CRC welcomes all students, regardless of their sobriety status [Source 1](https://counseling.uoregon.edu/collegiate-recovery).

2. **Counseling Services**: They provide assessment, co

In [16]:

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(annotation.text, f' [{index+1}]')
    
    #Get the File ID for the file in this citation
    file_id = annotation.file_citation.file_id

    #Find the URL or source for the file_id that OpenAI has
    citation = get_name_by_file_id(df_merge,file_id)

    # Add the citation to the list
    citations.append(f'[{index+1}] {citation}')

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n' + 'Sources' + '\n' + '\n'.join(citations)

In [17]:
print(message_content.value)

You asked about Lauren Eichler. She is the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon.

Lauren Eichler has an academic background that includes a PhD in Philosophy from the University of Oregon. Her research interests span several fields, including environmental ethics, genocide studies, Native American philosophies, and ethics in general. Before her current role, she taught philosophy at Old Dominion University, where she also served as the Director of Assessment for the Department of Philosophy and Religious Studies.

Eichler is deeply committed to academic advising and mentoring students, emphasizing the importance of liberal arts education and its impact on individuals and society. Her personal experiences and fondness for the UO community led her to return to the university, where she aims to make a meaningful contribution to student development [1].
[1] https://advising.uoregon.edu/-lauren-eichler
Sources
[

In [148]:
import time

formatted_string = message_content.value
delay = 0.01  # delay in seconds

for char in formatted_string:
    print(char, end='', flush=True)
    time.sleep(delay)

You're interested in learning about Lauren Eichler. She is currently the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon.

Lauren Eichler has an extensive academic background, having grown up in Baltimore, MD. She earned her PhD in Philosophy from the University of Oregon after spending a decade in higher education and research. Her areas of interest in philosophy include environmental ethics, genocide studies, and Native American philosophies. Before returning to UO, she taught philosophy at Old Dominion University and served as the Director of Assessment in their Department of Philosophy and Religious Studies. 

In her role at UO, Lauren is deeply committed to mentoring students and guiding them in their academic and professional journeys. She describes advising as a way to engage in meaningful conversations that support students both practically and existentially [1] [2].

If you'd like to know more about her work 